# Modeling kelp site viability

In [144]:
import pandas as pd
import os, os.path
import urllib
from sklearn.model_selection import train_test_split
import numpy as np

%matplotlib inline

In [77]:
dir_name = "../data"
pickles = '../data/pickles/'
gcs_bucket = "https://storage.googleapis.com/w210data/"


#if not os.path.isdir(dir_name):
#    os.mkdir(dir_name)
    
#def download_file(file_name):
#    full_path = os.path.join(dir_name, file_name)
#    if not os.path.exists(full_path):
#        urllib.request.urlretrieve (gcs_bucket+file_name, full_path)

#download_file(pickles+'grid_df_tenth.pkl')

In [173]:
grid_df = pd.read_pickle(pickles+'grid_df.pkl')
#keep polygon_id as identifier
polygon_id = grid_df.polygon_id
#extract only features relevant to predicting kelp viability
grid_df = grid_df[['biomass','depth','mean_sst','max_sst','min_sst','ndvi',\
                   'z_min_light','z_mixedl','floor_temp','viable','shoretype','shoretype2','aerial_kelp']]
#clean up categorical variables
grid_df.shoretype = grid_df.shoretype.apply(lambda x: x[0][1])
grid_df.shoretype2 = grid_df.shoretype2.apply(lambda x: x[0][1])
grid_df.aerial_kelp = grid_df.aerial_kelp.apply(lambda x: x[0][1])
print(grid_df.shape)
grid_df.head()

(491, 13)


,biomass,depth,mean_sst,max_sst,min_sst,ndvi,z_min_light,z_mixedl,floor_temp,viable,shoretype,shoretype2,aerial_kelp
0,0.0,-122.4,11.929242,14.013989,9.125995,0.008434,43.0,17.955626,8.584,False,Rocky Shores,exposed wave cut platforms in bedrock,False
1,0.0,-45.0,11.929242,14.013989,9.125995,0.008434,46.0,17.955626,8.156,True,Rocky Shores,gravel beaches,False
2,0.0,-29.6,11.929242,14.013989,9.125995,0.008434,28.0,17.955626,8.156,False,Rocky Shores,exposed rocky cliffs,False
3,0.0,-38.6,11.929242,14.013989,9.125995,0.008434,32.0,17.955626,8.156,False,Rocky Shores,exposed rocky cliffs,False
4,0.0,-58.2,11.929242,14.013989,9.125995,0.008434,42.0,17.955626,8.910,False,Rocky Shores,gravel beaches,False


In [174]:
#separate features from output classes
y = grid_df[['aerial_kelp']]
x = grid_df.drop(['aerial_kelp'], axis=1)

#create train/dev/test sets
x_train, x_dev, y_train, y_dev = train_test_split(x,y, test_size=0.3, shuffle=True)
x_dev, x_test, y_dev, y_test = train_test_split(x_dev,y_dev, test_size=0.33)
print(x_train.shape)
print(x_dev.shape)
print(x_test.shape)

(343, 12)
(99, 12)
(49, 12)


In [155]:
#create dummy variables for the categorical variables
shoretype = pd.get_dummies(grid_df['shoretype'])
shoretype2 = pd.get_dummies(grid_df['shoretype2'])

#remove categorical variables so we can scale the numeric vars
grid_df.drop(['shoretype','shoretype2'], axis=1, inplace=True)

#scale numeric variables


#add dummy variables for shoretype and shoretype2
grid_df = grid_df.join(shoretype).join(shoretype2)
print(grid_df.shape)
grid_df.head()

(491, 32)


,biomass,depth,mean_sst,max_sst,min_sst,ndvi,z_min_light,z_mixedl,floor_temp,viable,...,gravel beaches,mixed sand and gravel beaches,riprap,salt and brackish water marshes,salt marshes,sheltered man-made structures,sheltered rocky shores,sheltered tidal flats,vegetated low riverine banks,wave cut rocky platforms
0,0.0,-122.4,11.929242,14.013989,9.125995,0.008434,43.0,17.955626,8.584,False,...,0,0,0,0,0,0,0,0,0,0
1,0.0,-45.0,11.929242,14.013989,9.125995,0.008434,46.0,17.955626,8.156,True,...,1,0,0,0,0,0,0,0,0,0
2,0.0,-29.6,11.929242,14.013989,9.125995,0.008434,28.0,17.955626,8.156,False,...,0,0,0,0,0,0,0,0,0,0
3,0.0,-38.6,11.929242,14.013989,9.125995,0.008434,32.0,17.955626,8.156,False,...,0,0,0,0,0,0,0,0,0,0
4,0.0,-58.2,11.929242,14.013989,9.125995,0.008434,42.0,17.955626,8.910,False,...,1,0,0,0,0,0,0,0,0,0


0.42857142857142855